In [1]:
#Pull list of keys from S3
import boto3
import os
import gzip
import json
import string
import re
from datetime import date
import pandas as pd

s3 = boto3.client('s3')
a = s3.list_objects_v2(Bucket='lex-conversationlog-midschatbot-webui')

keys={}
x = 0
y = 0

#to pull consolidated logs
for i in a['Contents']:
    if i['Key'][-2:]=='gz':
        keys[x]=i['Key']
        x+=1

#to pull specific export log stream        
#for i in a['Contents']:
    #if i['Key'][14:50]=='ff291486-c7db-4c7b-ba32-d6eefd281210': #enter latest random string
        #keys[y]=i['Key']
       #y+=1
keys

{0: 'convlogs_test/1ec9db04-b961-442f-af9b-4fd7cf77fac4/aws-lex-MIDSChatbot_VA-MIDSChatbot_VA_WebUI-48-38495b662c0a54f7f4e72ec8f58bd47d/000000.gz',
 1: 'convlogs_test/1ec9db04-b961-442f-af9b-4fd7cf77fac4/aws-lex-MIDSChatbot_VA-MIDSChatbot_VA_WebUI-48-65b928756f73256c169e4d1244297216/000000.gz',
 2: 'convlogs_test/1ec9db04-b961-442f-af9b-4fd7cf77fac4/aws-lex-MIDSChatbot_VA-MIDSChatbot_VA_WebUI-48-a13f159b787f83675703e62f4bd8aebe/000000.gz',
 3: 'convlogs_test/1ec9db04-b961-442f-af9b-4fd7cf77fac4/aws-lex-MIDSChatbot_VA-MIDSChatbot_VA_WebUI-48-f71abb772a1a30a09e853f10fd05236c/000000.gz',
 4: 'convlogs_test/1ec9db04-b961-442f-af9b-4fd7cf77fac4/aws-lex-MIDSChatbot_VA-MIDSChatbot_VA_WebUI-49-0d1705b7a4ef408dcc3428a848b4fd37/000000.gz',
 5: 'convlogs_test/1ec9db04-b961-442f-af9b-4fd7cf77fac4/aws-lex-MIDSChatbot_VA-MIDSChatbot_VA_WebUI-49-28bbf6eb8fe91ce9bb87790d2a664fb9/000000.gz',
 6: 'convlogs_test/1ec9db04-b961-442f-af9b-4fd7cf77fac4/aws-lex-MIDSChatbot_VA-MIDSChatbot_VA_WebUI-49-560343b13

In [2]:
here = str('consolidated') #Set directory

In [ ]:
#download each log into files
z = []


for i in keys:
    a ='cmd /k aws s3api get-object --key '+ keys[i] + ' --bucket arn:aws:s3:us-east-1:098766315756:accesspoint/access-conversationlog w210/zipfile/'+here+'/log' +str(i)+'.gz'
    z.append(a)

for i in z:
    os.system(i)
    
print("Total ",len(z)," sessions logged in stream")


In [120]:
#Code to extract zip files into text files
log=[]

#to read from consolidated folder        
for i in range(len(keys)):
    fpath = 'w210/zipfile/'+here+'/log'+str(i)+'.gz'
    with gzip.open(fpath, 'rb') as f:
        data = f.read().decode('utf-8').split('\n')
        for i in data:
            log.append(i[25:])
        log.remove('')

#load json object
read_log={"all_events":[]}
a = read_log["all_events"]

#load daylog to json
#for i in log:
   # a.append(json.loads(i))

#load full to json
for i in log:
    a.append(json.loads(i))

a[0]

{'botName': 'MIDSChatbot_VA',
 'botAlias': 'MIDSChatbot_VA_WebUI',
 'botVersion': '48',
 'inputTranscript': 'agent',
 'botResponse': 'Sorry, can you please repeat that?',
 'alternativeIntents': [{'name': 'Application_AdmissionsCycle',
   'id': None,
   'nluIntentConfidence': '0.72',
   'slots': {},
   'slotDetails': None},
  {'name': 'Greeting',
   'id': None,
   'nluIntentConfidence': '0.64',
   'slots': {'UserName': None},
   'slotDetails': None},
  {'name': 'General_thumbs_up',
   'id': None,
   'nluIntentConfidence': '0.63',
   'slots': {},
   'slotDetails': None},
  {'name': 'General_thumbs_down',
   'id': None,
   'nluIntentConfidence': '0.63',
   'slots': {},
   'slotDetails': None}],
 'missedUtterance': True,
 'inputDialogMode': 'Text',
 'requestId': 'c1d1bb0c-f12d-47b5-8cec-7916804cdaf9',
 'userId': 'us-east-1:c58ed80e-8c7b-4cd0-a3f3-968c52195009',
 'sessionId': '2021-03-31T19:57:48.709Z-sVuGbmbY',
 'dialogState': 'ElicitIntent',
 'locale': 'en_US',
 'timestamp': '2021-03-31T1

In [8]:
sessionID={}
missedUtterance={}
popularIntent={}
botVersion = {}

for i in a:
    botVersion[i['botVersion']]= {}
    botVersion[i['botVersion']]['messageLen']=0
    botVersion[i['botVersion']]['missed']=0
    botVersion[i['botVersion']]['missedUtterance']={}
    botVersion[i['botVersion']]['sessionId']=[]
    botVersion[i['botVersion']]['sessioncount']=0
    botVersion[i['botVersion']]['avgLen']=0
    sessionID[i['sessionId']]= {}
    sessionID[i['sessionId']]['messageLen']=0
    sessionID[i['sessionId']]['missedUtterance']=0
    #sessionID[i['sessionId']]['timestamp']=i['timestamp'][:16]
    popularIntent[i['inputTranscript'].lower()]={}
    popularIntent[i['inputTranscript'].lower()]['count']=0
    if i['missedUtterance'] == True:
        missedUtterance[i['inputTranscript'].lower()]={}
        missedUtterance[i['inputTranscript'].lower()]['count']=0
        missedUtterance[i['inputTranscript'].lower()]['sessionID']=[]
        missedUtterance[i['inputTranscript'].lower()]['botVersion']=[]
                
for i in a:
    if i['botVersion'] in botVersion:
        botVersion[i['botVersion']]['messageLen']+=1
        botVersion[i['botVersion']]['sessionId'].append(i['sessionId'])
        if i['missedUtterance'] == True:
            botVersion[i['botVersion']]['missedUtterance'][i['inputTranscript'].lower()]=0
            botVersion[i['botVersion']]['missed'] += 1
    if i['sessionId'] in sessionID:
        sessionID[i['sessionId']]['messageLen']+=1
        if i['missedUtterance'] == True:
                sessionID[i['sessionId']]['missedUtterance']+=1
                missedUtterance[i['inputTranscript'].lower()]['count']+=1
                missedUtterance[i['inputTranscript'].lower()]['sessionID'].append(i['sessionId'])
                missedUtterance[i['inputTranscript'].lower()]['botVersion'].append(i['botVersion']+"-"+i['timestamp'][2:4]+i['timestamp'][5:7]+i['timestamp'][8:16])
                missedUtterance[i['inputTranscript'].lower()]['alternativeIntents']=i['alternativeIntents']
    if i['inputTranscript'].lower() in popularIntent:
        popularIntent[i['inputTranscript'].lower()]['count']+=1
for i in a:
    if i['missedUtterance'] == True:
            botVersion[i['botVersion']]['missedUtterance'][i['inputTranscript'].lower()]+=1

for i in missedUtterance:
    missedUtterance[i]['botVersion']=set(missedUtterance[i]['botVersion'])
    
for i in sessionID:
    if sessionID[i]['missedUtterance']==0:
        sessionID[i]['fallbackRate'] = 0
    else: 
        sessionID[i]['fallbackRate']=round(sessionID[i]['missedUtterance']/sessionID[i]['messageLen'],2)

for i in botVersion:
    botVersion[i]['sessionId']=set(botVersion[i]['sessionId'])
    botVersion[i]['sessioncount']=len((botVersion[i]['sessionId']))
    botVersion[i]['avgLen'] = botVersion[i]['messageLen']/botVersion[i]['sessioncount']
    if botVersion[i]['missed']==0:
        botVersion[i]['fallbackRate'] = 0
    else: 
        botVersion[i]['fallbackRate']=round(botVersion[i]['missed']/botVersion[i]['messageLen'],2)


In [118]:
df=pd.read_json(json.dumps(a))
len(df.userId.unique())
#df.userId.unique()


35

In [119]:
ignore_sessions = {'us-east-1:c58ed80e-8c7b-4cd0-a3f3-968c52195009':"Steve",'us-east-1:c1f93def-8f5c-43b9-99fa-b18a9e7e238d':"Steve",'us-east-1:4f3fc32b-37f0-4d79-a02b-8b9a1d4af85a':"Capstone",'us-east-1:3cfbd130-25b8-42f1-b381-2ed85dc8c819':"Capstone",'us-east-1:3b3f3344-25f8-48a2-886b-2094effb11df':"Joanna",'us-east-1:626782de-d368-459c-b35e-913e9a18a461':"Capstone",'us-east-1:6bfef357-8a94-4019-8974-9e1e9ce070ba':"Capstone",'us-east-1:7fba9d26-888c-4373-bd05-91e13e461c67':"Capstone",'us-east-1:a20ca141-28dc-4f25-ab20-aceccbeb799a':"Test",'us-east-1:4f0fac53-483d-4115-8e34-51af0195c8c2':"Capstone", 'us-east-1:779bf778-05fc-4f5a-a6fa-c8c444a804b8':"Capstone",'us-east-1:a7497e35-03e5-428f-aeb8-e6ebfedbedcb':"Capstone",'us-east-1:ad20e1a4-30cd-49b5-be44-07de880be7bc':"Capstone",'us-east-1:ae7a8501-edf6-4af2-9df2-4d93ab8373f5':"Capstone",'us-east-1:0eee7f3d-5592-4586-bf64-72ba20116afc':"Capstone",'us-east-1:1cb3d5a0-652b-4e2a-a8aa-60a98b979b0b':"Nicole"}
ignore = [ignore_sessions.keys()]

len(df[~df.userId.isin(ignore_sessions.keys())].userId.unique())
df=df[~df.userId.isin(ignore_sessions.keys())]
len(df.sessionId.unique())

22

In [90]:
for i in a:
    if i['userId'] not in ignore_sessions.keys():
        botVersion[i['botVersion']]= {}
        botVersion[i['botVersion']]['messageLen']=0
        botVersion[i['botVersion']]['missed']=0
        botVersion[i['botVersion']]['missedUtterance']={}
        botVersion[i['botVersion']]['sessionId']=[]
        #botVersion[i['botVersion']]['sessioncount']=0
        botVersion[i['botVersion']]['avgLen']=0
        sessionID[i['sessionId']]= {}
        sessionID[i['sessionId']]['messageLen']=0
        sessionID[i['sessionId']]['missedUtterance']=0
        #sessionID[i['sessionId']]['timestamp']=i['timestamp'][:16]
        popularIntent[i['inputTranscript'].lower()]={}
        popularIntent[i['inputTranscript'].lower()]['count']=0
        if i['missedUtterance'] == True:
            missedUtterance[i['inputTranscript'].lower()]={}
            missedUtterance[i['inputTranscript'].lower()]['count']=0
            missedUtterance[i['inputTranscript'].lower()]['sessionID']=[]
            missedUtterance[i['inputTranscript'].lower()]['botVersion']=[]
                
for i in a:
    if i['userId'] not in ignore_sessions.keys():
        if i['botVersion'] in botVersion:
            botVersion[i['botVersion']]['messageLen']+=1
            botVersion[i['botVersion']]['sessionId'].append(i['sessionId'])
            if i['missedUtterance'] == True:
                botVersion[i['botVersion']]['missedUtterance'][i['inputTranscript'].lower()]=0
                botVersion[i['botVersion']]['missed'] += 1
        if i['sessionId'] in sessionID:
            sessionID[i['sessionId']]['messageLen']+=1
            if i['missedUtterance'] == True:
                    sessionID[i['sessionId']]['missedUtterance']+=1
                    missedUtterance[i['inputTranscript'].lower()]['count']+=1
                    missedUtterance[i['inputTranscript'].lower()]['sessionID'].append(i['sessionId'])
                    missedUtterance[i['inputTranscript'].lower()]['botVersion'].append(i['botVersion']+"-"+i['timestamp'][2:4]+i['timestamp'][5:7]+i['timestamp'][8:16])
                    missedUtterance[i['inputTranscript'].lower()]['alternativeIntents']=i['alternativeIntents']
        if i['inputTranscript'].lower() in popularIntent:
            popularIntent[i['inputTranscript'].lower()]['count']+=1
for i in a:
    if i['userId'] not in ignore_sessions.keys():
        if i['missedUtterance'] == True:
                botVersion[i['botVersion']]['missedUtterance'][i['inputTranscript'].lower()]+=1

for i in missedUtterance:
    missedUtterance[i]['botVersion']=set(missedUtterance[i]['botVersion'])
    
for i in sessionID:
    if sessionID[i]['missedUtterance']==0:
        sessionID[i]['fallbackRate'] = 0
    else: 
        sessionID[i]['fallbackRate']=round(sessionID[i]['missedUtterance']/sessionID[i]['messageLen'],2)

for i in botVersion:
    botVersion[i]['sessionId']=len(set(botVersion[i]['sessionId']))
    #botVersion[i]['sessioncount']=len((botVersion[i]['sessionId']))
    botVersion[i]['avgLen'] = botVersion[i]['messageLen']/botVersion[i]['sessionId']
    if botVersion[i]['missed']==0:
        botVersion[i]['fallbackRate'] = 0
    else: 
        botVersion[i]['fallbackRate']=round(botVersion[i]['missed']/botVersion[i]['messageLen'],2)


In [68]:
df[df.userId=='us-east-1:f950d378-c419-4472-b5bc-2b3d6bf05312'].sort_values(by = "botVersion")

,botName,botAlias,botVersion,inputTranscript,botResponse,alternativeIntents,missedUtterance,inputDialogMode,requestId,userId,...,locale,timestamp,developerOverride,sessionAttributes,messageVersion,intent,nluIntentConfidence,slots,responseCard,slotToElicit
982,MIDSChatbot_VA,MIDSChatbot_VA_WebUI,38,Where do I go next?,"Sorry, can you please repeat that?","[{'name': 'MainMenu', 'id': None, 'nluIntentCo...",True,Text,9a424451-4c74-401f-96cb-9cf5725dce55,us-east-1:f950d378-c419-4472-b5bc-2b3d6bf05312,...,en_US,2021-03-20 21:45:58.692000+00:00,False,{'userAgent': 'Mozilla/5.0 (Linux; Android 11;...,1,NaN,NaN,NaN,NaN,NaN
983,MIDSChatbot_VA,MIDSChatbot_VA_WebUI,38,What now?,"Sorry, can you please repeat that?","[{'name': 'MainMenu', 'id': None, 'nluIntentCo...",True,Text,18a51d9f-9886-4eaf-9985-55aed002a594,us-east-1:f950d378-c419-4472-b5bc-2b3d6bf05312,...,en_US,2021-03-20 21:46:08.342000+00:00,False,{'userAgent': 'Mozilla/5.0 (Linux; Android 11;...,1,NaN,NaN,NaN,NaN,NaN
984,MIDSChatbot_VA,MIDSChatbot_VA_WebUI,38,Bye,"Sorry, I could not understand. Goodbye.","[{'name': 'Greeting', 'id': None, 'nluIntentCo...",True,Text,8b6863b6-8316-4397-b5fb-12d3ab541fea,us-east-1:f950d378-c419-4472-b5bc-2b3d6bf05312,...,en_US,2021-03-20 21:46:31.401000+00:00,False,{'userAgent': 'Mozilla/5.0 (Linux; Android 11;...,1,NaN,NaN,NaN,NaN,NaN
985,MIDSChatbot_VA,MIDSChatbot_VA_WebUI,38,Walabi,"Nice to meet you, Walabi. Just so you know, th...","[{'name': 'AMAZON.FallbackIntent', 'id': None,...",False,Text,97cc5816-2060-4a9d-80b3-dd2ecb1a0385,us-east-1:f950d378-c419-4472-b5bc-2b3d6bf05312,...,en_US,2021-03-20 21:45:23.968000+00:00,False,{'userAgent': 'Mozilla/5.0 (Linux; Android 11;...,1,Greeting,1.0,{'UserName': 'Walabi'},{'genericAttachments': [{'title': 'Current MID...,NaN
986,MIDSChatbot_VA,MIDSChatbot_VA_WebUI,38,Other,"There are a number of factual, less conversati...","[{'name': 'General_MIDSsummary', 'id': None, '...",False,Text,5c5d4173-22d5-4e34-b6a2-9628a87fec47,us-east-1:f950d378-c419-4472-b5bc-2b3d6bf05312,...,en_US,2021-03-20 21:45:35.787000+00:00,False,{'userAgent': 'Mozilla/5.0 (Linux; Android 11;...,1,Other,1.0,{},NaN,NaN
987,MIDSChatbot_VA,MIDSChatbot_VA_WebUI,38,Hi,Hi! What is your name?,"[{'name': 'Application_GeneralRequirements', '...",False,Text,6c8ce4c7-2c40-402a-b8c9-c23a278665bb,us-east-1:f950d378-c419-4472-b5bc-2b3d6bf05312,...,en_US,2021-03-20 21:45:14.540000+00:00,False,{'userAgent': 'Mozilla/5.0 (Linux; Android 11;...,1,Greeting,1.0,{'UserName': None},NaN,UserName
988,MIDSChatbot_VA,MIDSChatbot_VA_WebUI,38,Sounds good,"Sorry, can you please repeat that?","[{'name': 'MIDS_Duration', 'id': None, 'nluInt...",True,Text,373a9181-3d9d-4547-842d-34e0924c746a,us-east-1:f950d378-c419-4472-b5bc-2b3d6bf05312,...,en_US,2021-03-20 21:45:47.500000+00:00,False,{'userAgent': 'Mozilla/5.0 (Linux; Android 11;...,1,NaN,NaN,NaN,NaN,NaN
989,MIDSChatbot_VA,MIDSChatbot_VA_WebUI,38,Back,"Sorry, can you please repeat that?","[{'name': 'Greeting', 'id': None, 'nluIntentCo...",True,Text,f7c8069a-c698-4c6b-90d2-a14379d2fdd6,us-east-1:f950d378-c419-4472-b5bc-2b3d6bf05312,...,en_US,2021-03-20 21:46:18.947000+00:00,False,{'userAgent': 'Mozilla/5.0 (Linux; Android 11;...,1,NaN,NaN,NaN,NaN,NaN
990,MIDSChatbot_VA,MIDSChatbot_VA_WebUI,38,Help!,"Sorry, can you please repeat that?","[{'name': 'Networking', 'id': None, 'nluIntent...",True,Text,152a4292-42b3-40e5-af69-e0677fb82f1c,us-east-1:f950d378-c419-4472-b5bc-2b3d6bf05312,...,en_US,2021-03-20 21:46:24.715000+00:00,False,{'userAgent': 'Mozilla/5.0 (Linux; Android 11;...,1,NaN,NaN,NaN,NaN,NaN


In [112]:
botVersion

{'48': {'messageLen': 45,
  'missed': 16,
  'missedUtterance': {'what is the hardest class in mids?': 2,
   'average salary': 1,
   'you guys are awesome': 1,
   'where is berkeley located': 1,
   'what career services can i get from mids': 1,
   'mids career service': 1,
   'do i get to meet my classmates in person?': 1,
   'berkely address': 1,
   'tuition': 1,
   'what is your name': 1,
   'i hope you win the capstone awared': 1,
   'what ist he addres of berkeley': 1,
   'how are you?': 1,
   'are all classes remote?': 1,
   'nice': 1},
  'sessionId': 5,
  'avgLen': 9.0,
  'fallbackRate': 0.36},
 '49': {'messageLen': 84,
  'missed': 21,
  'missedUtterance': {'impressive': 1,
   'airbnb': 1,
   'nope': 1,
   'kevin promised me this would work': 2,
   'tell me about the alumni network': 1,
   'why are you asking me to "repeat that" if repeating the question does not increase the liklihood of an answer triggering?': 1,
   'can i speak to someone?': 1,
   'ok...': 1,
   'return': 2,
  

In [114]:
df=pd.read_json(json.dumps(botVersion))
df=df.T
df = df.sort_index()
df.messageLen = df.messageLen.astype(int)
df.missed =df.missed.astype(int)
df.fallbackRate =df.fallbackRate.astype(float)
df.avgLen =df.avgLen.astype(float)
df.sessioncount =df.sessionId.astype(int)
print(df.dtypes)


early = df[df.index<31]
interim = df[(df.index>30)&(df.index<41)]
final = df[(df.index>40)]
print(early[["messageLen","missed","avgLen","sessioncount","fallbackRate"]].describe())
print(interim[["messageLen","missed","avgLen","sessioncount","fallbackRate"]].describe())
print(final[["messageLen","missed","avgLen","sessioncount","fallbackRate"]].describe())
print(df[["messageLen","missed","avgLen","sessioncount","fallbackRate"]].describe())
print(df[["messageLen","missed","avgLen","sessioncount","fallbackRate"]])

df

messageLen           int32
missed               int32
missedUtterance     object
sessionId           object
avgLen             float64
fallbackRate       float64
sessioncount         int32
dtype: object
       messageLen    missed     avgLen  sessioncount  fallbackRate
count    4.000000  4.000000   4.000000      4.000000      4.000000
mean    28.500000  3.750000  18.666667      2.000000      0.162500
std     17.406895  2.753785  18.292378      0.816497      0.094296
min      5.000000  1.000000   2.500000      1.000000      0.050000
25%     20.750000  1.750000   7.125000      1.750000      0.110000
50%     32.500000  3.500000  14.083333      2.000000      0.165000
75%     40.250000  5.500000  25.625000      2.250000      0.217500
max     44.000000  7.000000  44.000000      3.000000      0.270000
       messageLen     missed     avgLen  sessioncount  fallbackRate
count    8.000000   8.000000   8.000000      8.000000       8.00000
mean    34.875000  11.875000  18.312500      2.250000     

,messageLen,missed,missedUtterance,sessionId,avgLen,fallbackRate,sessioncount
26,5,1,{'test scores': 1},2,2.500000,0.20,2
27,44,2,"{'how long is my score valid for?': 1, 'nah': 1}",1,44.000000,0.05,1
29,26,7,{'i live in china. is the experience still the...,3,8.666667,0.27,3
30,39,5,"{'i dont know': 1, '5': 1, 'none': 1, '10 hour...",2,19.500000,0.13,2
31,55,16,"{'test': 1, 'what can i ask': 2, 'applications...",2,27.500000,0.29,2
32,59,23,"{'return': 1, 'got it': 1, 'ummm': 1, 'link': ...",6,9.833333,0.39,6
33,9,3,"{'capstone team': 1, 'capstone': 2}",1,9.000000,0.33,1
35,35,12,"{'are the parties fun?': 1, 'thanks!': 1, 'ok,...",1,35.000000,0.34,1
36,50,12,"{'official language is english': 1, 'electives...",3,16.666667,0.24,3
37,15,5,"{'career services': 2, 'can i submit a letter ...",2,7.500000,0.33,2


In [ ]:
sessionID
df=pd.read_json(json.dumps(sessionID))
df=df.T
df.describe()

In [ ]:
df

In [ ]:
missedUtterance

In [ ]:
#list of missed utterance and botVersion
missedList=[]
for i in missedUtterance:
    missedList.append((i,missedUtterance[i]['botVersion']))
missedList=sorted(missedList)
for i in missedList:
    print(i[0],';',i[1])


In [ ]:
#find session ID for deeper analysis
missedUtterance['is this teh same as an in person degree from berkeley']

In [ ]:
#Get list of input transcripts by count
def getKey(item):
    return item[1]

priorities=[]
for intent in popularIntent:
    priorities.append((intent,popularIntent[intent]['count']))
priorities.sort()   
priorities

sorted(priorities, key=getKey,reverse=True)
